In [10]:
%reload_ext autotime
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)
from pprint import pprint
import json5 as json # This is a more forgiving JSON parser that can handle comments, single quotes, and trailing commas
from glob import glob
files = glob("videos/*.json")
print(len(files))

294
time: 6.85 ms (started: 2025-10-17 16:45:55 +13:00)


In [ ]:
from transformers import Qwen3OmniMoeForConditionalGeneration, Qwen3OmniMoeProcessor
from qwen_omni_utils import process_mm_info
import torch
MODEL_PATH = "Qwen/Qwen3-Omni-30B-A3B-Instruct"
# Uses about 78GB VRAM
model = Qwen3OmniMoeForConditionalGeneration.from_pretrained(
    MODEL_PATH,
    dtype=torch.bfloat16,
    device_map="auto",
    attn_implementation="flash_attention_2",
)
model.disable_talker()
processor = Qwen3OmniMoeProcessor.from_pretrained(MODEL_PATH)

Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'interleaved', 'mrope_interleaved', 'mrope_section'}
Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'interleaved', 'mrope_section'}
You are attempting to use Flash Attention 2 without specifying a torch dtype. This might lead to unexpected behaviour


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.


time: 1min 51s (started: 2025-10-17 16:19:48 +13:00)


In [11]:
with open("videos/Video by livjowen [DCHwWHnPu89].info.json") as f:
    data = json.load(f)
display(pd.json_normalize(data))
print(data["description"])

,id,formats,title,description,duration,timestamp,uploader_id,uploader,channel,like_count,comment_count,comments,thumbnails,webpage_url,webpage_url_basename,webpage_url_domain,extractor,extractor_key,thumbnail,display_id,fulltitle,duration_string,upload_date,epoch,format,format_id,ext,protocol,format_note,filesize_approx,tbr,width,height,resolution,dynamic_range,vcodec,vbr,aspect_ratio,acodec,abr,asr,_type,http_headers.Referer,_version.version,_version.release_git_head,_version.repository
0,DCHwWHnPu89,"[{'format_id': 'dash-792918992914787ad', 'ext': 'm4a', 'tbr': 87.185, 'asr': 44100, 'format_note...",Video by livjowen,✨ Let’s talk about when (and when not) to use time-outs with our little ones. This can be a tric...,89.733,1731092557,51701748602,Olivia J Owen,livjowen,13061,417,"[{'author': 'patricia.c.loranca', 'author_id': '9475602362', 'id': '18060687419408755', 'text': ...",[{'url': 'https://instagram.fwlg3-2.fna.fbcdn.net/v/t51.2885-15/504250553_1780875225810106_90077...,https://www.instagram.com/reel/DCHwWHnPu89/?hl=en,DCHwWHnPu89,instagram.com,Instagram,Instagram,https://instagram.fwlg3-2.fna.fbcdn.net/v/t51.2885-15/504250553_1780875225810106_900771867193260...,DCHwWHnPu89,Video by livjowen,1:29,20241108,1760562270,dash-1477449796896131_mpd_qe=_v - 1080x1920 (DASH video)+dash-792918992914787ad - audio only (DA...,dash-1477449796896131_mpd_qe=_v+dash-792918992914787ad,mp4,https+https,DASH video+DASH audio,34212018,3050.117,1080,1920,1080x1920,SDR,vp09.00.40.08.00.01.01.01.00,2962.932,0.56,mp4a.40.5,87.185,44100,video,https://www.instagram.com/,2025.10.14,a98e7f9f58a9492d2cb216baa59c890ed8ce02f3,yt-dlp/yt-dlp


✨ Let’s talk about when (and when not) to use time-outs with our little ones. This can be a tricky topic, so here’s a quick guide I follow with my toddlers:

💥 When a Time-Out Is Helpful:
Time-outs are for moments when a child’s behavior escalates beyond what they can control—like screaming, hitting, or refusing to listen. These are signs that their emotions have taken over, and they need a moment to calm down before they’re able to hear or respond to us. Time-outs provide a safe space for them to reset and regain control of their feelings. 💛

🌱 When No Time-Out Is Needed:
On the flip side, when a child is upset but can still listen, express their feelings, or engage in communication, that’s actually a huge win. This is when they’re learning how to work through emotions in real time, and it’s a great opportunity for us to be there, help them process, and validate their feelings without needing a break. ✨

Using time-outs thoughtfully helps kids feel safe and understood. They’re not pun

In [6]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "video",
                "video": "videos/Video by livjowen [DCHwWHnPu89].mp4",
                "max_pixels": 360 * 420,
            },
            {
                "type": "text",
                "text": f"""This is a video downloaded from Instagram. Here's the description of the video: {data['description']}. Taking into account this description, and the video, extract the following information, in JSON format:
                    description: What is happening in the video? Provide a detailed description of the actions, context, and any notable elements present in the video.
                    transcript: If there is any spoken content in the video, transcribe it accurately. If there is no spoken content, indicate "No spoken content".
                    tone: What is the overall tone or mood of the video? Is it humorous, serious, educational, emotional, etc.?
                    timeout: Is this video talking about timeout for children (as a parenting punishement strategy)? True or False?
                    language: What language is this video in?
                    marketing: Is this video promoting or advertising any product, service, brand, or organization? If so, what is it? Otherwise, indicate "No marketing content".
                    ASD: Is this video specifically targeted towards individuals with Autism Spectrum Disorder (ASD) or does it contain content that is particularly relevant or beneficial for this audience? True or False?
                    ADHD: Is this video specifically targeted towards individuals with Attention Deficit Hyperactivity Disorder (ADHD) or does it contain content that is particularly relevant or beneficial for this audience? True or False?
                    anxiety: Is this video specifically targeted towards individuals dealing with anxiety or does it contain content that is particularly relevant or beneficial for this audience? True or False?
                    job: For the main speaker, what is their job or profession? If it is not mentioned in the video, indicate "No job information". One or more (comma separated) of the following: therapist, psychologist, pediatrician, doctor, nurse, teacher, professor, social worker, counselor, coach, influencer, content creator?
                    sentiment: Does this video recommend timeout, discourage it, or is it neutral? One of negative, neutral or positive
                    criticism: If the video is critical of timeout, what are the main criticisms mentioned? One or more of:
                        Concerns about physical safety – child
                        Concerns about emotional or social development or wellbeing - child
                        Concerns about relational quality – parent-child
                        Concerns about practicality or usability
                        Concerns about litigation
                    alternative_strategies: Does the video mention any alternative strategies to timeout? If so, what are they?
                    usefulness: Rate the overall usefulness of the video on a scale from 1 to 10, where 1 is not useful at all and 10 is extremely useful.
                    misleading: Rate the extent to which the video contains misleading or inaccurate information on a scale from 1 to 10, where 1 is not misleading at all and 10 is extremely misleading.
                    quality: Rate the overall quality of the video on a scale from 1 to 10, where 1 is very poor quality and 10 is excellent quality.
                    personal_experience: Does the speaker mention any personal experience with timeout? If so, briefly summarize it.

                    Do not include comments in your JSON response. Only respond with the JSON object. Make sure the JSON is valid
                """
            },
        ],
    }
]
# Set whether to use audio in video
USE_AUDIO_IN_VIDEO = True

# Preparation for inference
text = processor.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
audios, images, videos = process_mm_info(messages, use_audio_in_video=USE_AUDIO_IN_VIDEO)
inputs = processor(text=text, 
                   audio=audios, 
                   images=images, 
                   videos=videos, 
                   return_tensors="pt", 
                   padding=True, 
                   use_audio_in_video=USE_AUDIO_IN_VIDEO)
inputs = inputs.to(model.device).to(model.dtype)

# Inference: Generation of the output text and audio
text_ids, audio = model.generate(**inputs, 
                                 thinker_return_dict_in_generate=True,
                                 use_audio_in_video=USE_AUDIO_IN_VIDEO)

text = processor.batch_decode(text_ids.sequences[:, inputs["input_ids"].shape[1] :],
                              skip_special_tokens=True,
                              clean_up_tokenization_spaces=False)[0]
print(text)

/mnt/timeout/.venv/lib/python3.12/site-packages/qwen_omni_utils/v2_5/audio_process.py:85: UserWarning: PySoundFile failed. Trying audioread instead.
  librosa.load(
/mnt/timeout/.venv/lib/python3.12/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


```json
{
    "description": "The video is a parenting advice segment where a woman discusses the appropriate use of time-outs for toddlers. She begins by explaining that a time-out is not a punishment but a break from a situation. She then provides two contrasting scenarios: one where a child is upset but can communicate and does not need a time-out, and another where a child is screaming and kicking and needs a break to calm down. The video includes clips of her interacting with her young children in these situations.",
    "transcript": "There's been a lot of discussion and a lot of controversy surrounding time out and when and if they are appropriate. Now I've said it before, time out is not the punishment. It is literally just time out from the situation. We are just gonna take a break. So, I just wanna take a minute and show you a comparison of a situation where I would use time out and a situation where I don't need to use time out. Come here. Whose Spidey is this? This is Hudso

In [7]:
text = text.replace("```json", "").replace("```", "").strip()
result = json.loads(text)
pprint(result)

{'ADHD': 'False',
 'ASD': 'False',
 'alternative_strategies': 'For a child who is upset but can communicate, the '
                           'speaker suggests sitting with them, validating '
                           'their feelings, and helping them process the '
                           'situation.',
 'anxiety': 'False',
 'criticism': [],
 'description': 'The video is a parenting advice segment where a woman '
                'discusses the appropriate use of time-outs for toddlers. She '
                'begins by explaining that a time-out is not a punishment but '
                'a break from a situation. She then provides two contrasting '
                'scenarios: one where a child is upset but can communicate and '
                'does not need a time-out, and another where a child is '
                'screaming and kicking and needs a break to calm down. The '
                'video includes clips of her interacting with her young '
                'children in these 